# Chapter 13: Agent Memory - Solutions
**From: Zero to AI Agent**

⚠️ **Try the exercises in the main notebook first before viewing solutions!**

In [ ]:
!pip install -q -r requirements.txt

from dotenv import load_dotenv
load_dotenv()

---
## Exercise 13.1.1 Solution: Memory Classification

### Scenario Analysis

**1. "Let's talk about my upcoming trip to Paris."**
- **Classification: Both Short-Term and Long-Term**
- Short-term: Current conversation topic needs to be in active context
- Long-term: The fact that user is planning a Paris trip is worth storing for future sessions

**2. "I'm allergic to peanuts."**
- **Classification: Long-Term (primarily)**
- This is critical safety/health information that should persist indefinitely
- Every future conversation involving food should have access to this fact

**3. "What did I just say about the hotel?"**
- **Classification: Short-Term**
- The word "just" signals immediate context
- This is a reference to current conversation, not something to store persistently

**4. Intermediate calculation during math problem**
- **Classification: Short-Term Only**
- Working memory for current task only
- No value after the problem is solved

**5. "Remember, I prefer bullet points over long paragraphs."**
- **Classification: Long-Term (primarily)**
- Persistent user preference that should affect ALL future interactions
- The word "remember" explicitly signals storage request

**6. "Today's weather is really nice."**
- **Classification: Short-Term Only**
- Ephemeral information with no future value
- Weather will be different tomorrow

---
## Exercise 13.1.2 Solution: Design a Memory Schema

### Personal Finance Assistant - Memory Schema

**1. Short-Term Memory:**
```
short_term_memory:
  conversation_history:
    - messages[]
    - current_topic
  working_state:
    - current_calculations
    - draft_budget
    - pending_actions[]
```

**2. Long-Term Memory:**
```
long_term_memory:
  user_profile:
    - name, preferred_currency, timezone
  financial_profile:
    - income_sources[]
    - recurring_expenses[]
    - accounts[]
    - financial_goals[]
  preferences:
    - budget_style
    - notification_preferences
```

**3. Transfer Rules:**
- Automatic: income, recurring bills, goals, preferences
- Prompted: budgets created, major financial changes

**4. Never Store:**
- Bank account numbers, passwords, SSN, full credit card numbers
- Store ranges instead of exact salary amounts

---
## Exercise 13.1.3 Solution: Memory Retrieval Strategy

### Multi-Stage Retrieval Algorithm

**Stage 1: Critical Safety (Always Include)**
- Query: allergies, dietary restrictions
- No limit, all matches included

**Stage 2: Semantic Search**
- Embed query, search food/dining memories
- Top 10 by similarity > 0.7

**Stage 3: Recent Context**
- Filter: last 90 days, dining category
- Sort by recency × relevance

**Stage 4: Location/Logistics**
- 2-3 most recent location facts

**Conflict Resolution:**
- Recency wins for preferences
- Safety info never expires
- Explicit updates override old facts

**Output Format:**
```
CRITICAL: Vegetarian, Shellfish allergy
PREFERENCES (High): Outdoor seating, quiet
PREFERENCES (Medium): Italian cuisine
RECENT: Tried Olive Garden - "just okay"
LOCATION: Downtown Seattle, no car
```

---
## Exercise 13.2.1 Solution: Basic Memory Implementation

In [ ]:
from openai import OpenAI

client = OpenAI()
conversation_history = []

# Add system prompt
conversation_history.append({
    "role": "system",
    "content": "You are a helpful math tutor."
})

def chat(user_message):
    conversation_history.append({"role": "user", "content": user_message})
    
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=conversation_history
    )
    
    assistant_message = response.choices[0].message.content
    conversation_history.append({"role": "assistant", "content": assistant_message})
    
    print(f"[Total messages: {len(conversation_history)}]")
    return assistant_message

# Test
print("User: What is 5 + 3?")
print(f"Assistant: {chat('What is 5 + 3?')}\n")

print("User: Now multiply that by 2")
print(f"Assistant: {chat('Now multiply that by 2')}\n")

print("User: What were we calculating?")
print(f"Assistant: {chat('What were we calculating?')}")

---
## Exercise 13.2.2 Solution: Smart Token-Aware Trimming

In [ ]:
from openai import OpenAI

client = OpenAI()

class TokenAwareMemory:
    def __init__(self, system_prompt, max_tokens=500):
        self.max_tokens = max_tokens
        self.messages = [{"role": "system", "content": system_prompt}]
    
    def estimate_tokens(self, text):
        return int(len(text.split()) * 1.3)
    
    def total_tokens(self):
        return sum(self.estimate_tokens(m["content"]) for m in self.messages)
    
    def trim_if_needed(self):
        while self.total_tokens() > self.max_tokens and len(self.messages) > 2:
            for i, msg in enumerate(self.messages):
                if msg["role"] != "system":
                    removed = self.messages.pop(i)
                    print(f"⚠️ TRIMMED: {msg['role']} message ({self.estimate_tokens(removed['content'])} tokens)")
                    break
    
    def chat(self, user_message):
        self.messages.append({"role": "user", "content": user_message})
        self.trim_if_needed()
        
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=self.messages
        )
        
        reply = response.choices[0].message.content
        self.messages.append({"role": "assistant", "content": reply})
        self.trim_if_needed()
        
        print(f"[Tokens: {self.total_tokens()}/{self.max_tokens}]")
        return reply

# Test
memory = TokenAwareMemory("You are a storyteller.", max_tokens=500)
print(memory.chat("Tell me a short story about a knight.")[:100] + "...")

---
## Exercise 13.2.3 Solution: Conversation Analytics

In [ ]:
from datetime import datetime
from openai import OpenAI

client = OpenAI()

class AnalyticsMemory:
    def __init__(self, system_prompt=None):
        self.messages = []
        self.start_time = None
        self.last_activity = None
        
        if system_prompt:
            self.messages.append({
                "role": "system", "content": system_prompt,
                "timestamp": datetime.now(), "char_count": len(system_prompt)
            })
    
    def add_message(self, role, content):
        now = datetime.now()
        if self.start_time is None:
            self.start_time = now
        self.last_activity = now
        self.messages.append({"role": role, "content": content, "timestamp": now, "char_count": len(content)})
    
    def chat(self, user_message):
        self.add_message("user", user_message)
        api_messages = [{"role": m["role"], "content": m["content"]} for m in self.messages]
        response = client.chat.completions.create(model="gpt-3.5-turbo", messages=api_messages)
        reply = response.choices[0].message.content
        self.add_message("assistant", reply)
        return reply
    
    def get_stats(self):
        counts = {"user": 0, "assistant": 0, "system": 0}
        lengths = {"user": [], "assistant": [], "system": []}
        longest = {"content": "", "role": "", "length": 0}
        
        for msg in self.messages:
            counts[msg["role"]] += 1
            lengths[msg["role"]].append(msg["char_count"])
            if msg["char_count"] > longest["length"]:
                longest = {"content": msg["content"][:100], "role": msg["role"], "length": msg["char_count"]}
        
        avg = {r: sum(l)/len(l) if l else 0 for r, l in lengths.items()}
        duration = (self.last_activity - self.start_time).total_seconds() if self.start_time else 0
        
        return {"total": len(self.messages), "counts": counts, "avg_lengths": avg, "longest": longest, "duration": duration}

# Test
memory = AnalyticsMemory("You are concise.")
memory.chat("Hello!")
memory.chat("Explain machine learning.")
print(memory.get_stats())

---
## Exercise 13.3.1 Solution: Basic Summarization

In [ ]:
from openai import OpenAI

client = OpenAI()

def summarize_messages(messages):
    """Summarize a list of conversation messages."""
    formatted = "\n\n".join(f"{m['role'].upper()}: {m['content']}" for m in messages)
    
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{
            "role": "user",
            "content": f"""Summarize this conversation, preserving key facts and decisions:

{formatted}

Provide a concise summary:"""
        }],
        max_tokens=300
    )
    return response.choices[0].message.content

# Test
test_messages = [
    {"role": "user", "content": "I want to plan a vacation to Italy in June."},
    {"role": "assistant", "content": "Italy in June is wonderful! Cities or coastal areas?"},
    {"role": "user", "content": "Rome and coastal. 10 days, $3000 budget. I'm vegetarian."}
]

print("SUMMARY:")
print(summarize_messages(test_messages))

---
## Exercise 13.3.2 Solution: Triggered Summarization

In [ ]:
from openai import OpenAI

client = OpenAI()

class SmartMemory:
    def __init__(self, system_prompt=None, max_messages=15, keep_recent=5):
        self.max_messages = max_messages
        self.keep_recent = keep_recent
        self.messages = []
        if system_prompt:
            self.messages.append({"role": "system", "content": system_prompt})
    
    def _generate_summary(self, messages):
        text = "\n".join(f"{m['role'].upper()}: {m['content']}" for m in messages)
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": f"Summarize concisely:\n\n{text}"}],
            max_tokens=250
        )
        return response.choices[0].message.content
    
    def _maybe_summarize(self):
        conversation = [m for m in self.messages if m["role"] in ("user", "assistant")]
        if len(conversation) <= self.max_messages:
            return
        
        to_summarize = conversation[:-self.keep_recent]
        to_keep = conversation[-self.keep_recent:]
        summary = self._generate_summary(to_summarize)
        
        system_prompt = [m for m in self.messages if m["role"] == "system" and "Summary:" not in m["content"]]
        self.messages = system_prompt + [{"role": "system", "content": f"Summary: {summary}"}] + to_keep
        
        print(f"\n🔄 SUMMARIZED {len(to_summarize)} messages\n")
    
    def chat(self, user_input):
        self.messages.append({"role": "user", "content": user_input})
        self._maybe_summarize()
        
        response = client.chat.completions.create(model="gpt-3.5-turbo", messages=self.messages)
        reply = response.choices[0].message.content
        self.messages.append({"role": "assistant", "content": reply})
        return reply

# Test
memory = SmartMemory("You are a travel assistant.", max_messages=10, keep_recent=4)
for q in ["I want to visit Japan.", "When should I go?", "How expensive?", "What cities?", "Tell me about Tokyo.", "What about Kyoto?"]:
    print(f"User: {q}")
    print(f"AI: {memory.chat(q)[:80]}...\n")

---
## Exercise 13.3.3 Solution: Domain-Specific Medical Summaries

In [ ]:
from openai import OpenAI
import json

client = OpenAI()

class MedicalConsultationMemory:
    def __init__(self):
        self.messages = [{"role": "system", "content": "You are a medical consultation assistant."}]
        self.medical_summary = {"symptoms": [], "duration": None, "medications": [], "recommendations": []}
    
    def _extract_medical_info(self, messages):
        text = "\n".join(f"{m['role'].upper()}: {m['content']}" for m in messages)
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": f"""Extract medical info as JSON:
{{"symptoms": [], "duration": null, "medications": [], "recommendations": []}}

{text}

JSON:"""}],
            max_tokens=300
        )
        try:
            return json.loads(response.choices[0].message.content.strip())
        except:
            return None
    
    def _update_summary(self, data):
        if not data:
            return
        for s in data.get("symptoms", []):
            if s not in self.medical_summary["symptoms"]:
                self.medical_summary["symptoms"].append(s)
        if data.get("duration"):
            self.medical_summary["duration"] = data["duration"]
        for m in data.get("medications", []):
            if m not in self.medical_summary["medications"]:
                self.medical_summary["medications"].append(m)
    
    def chat(self, user_input):
        self.messages.append({"role": "user", "content": user_input})
        response = client.chat.completions.create(model="gpt-3.5-turbo", messages=self.messages)
        reply = response.choices[0].message.content
        self.messages.append({"role": "assistant", "content": reply})
        
        extracted = self._extract_medical_info(self.messages[-4:])
        self._update_summary(extracted)
        return reply
    
    def get_medical_summary(self):
        return self.medical_summary

# Test
consult = MedicalConsultationMemory()
consult.chat("I've had headaches for a week, taking ibuprofen.")
print("Medical Summary:", consult.get_medical_summary())

---
## Exercise 13.4.1 Solution: Basic Entity Extraction

In [ ]:
from openai import OpenAI
import json

client = OpenAI()

def extract_entities(message):
    """Extract entities from a message using LLM."""
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{
            "role": "user",
            "content": f"""Extract entities as JSON:
{{"entities": [{{"name": "...", "type": "person|org|project|location", "info": "..."}}]}}

Message: {message}
JSON:"""
        }],
        max_tokens=200
    )
    
    try:
        return json.loads(response.choices[0].message.content)["entities"]
    except:
        return []

# Test
message = "John from marketing wants to discuss the Phoenix project with the Tokyo team."
entities = extract_entities(message)

print("Extracted Entities:")
for e in entities:
    print(f"  - {e['name']} ({e['type']}): {e.get('info', 'N/A')}")

---
## Exercise 13.4.2 Solution: Entity Memory Class

In [ ]:
class EntityMemory:
    def __init__(self):
        self.entities = {}
    
    def update(self, name, entity_type, fact=None):
        key = name.lower()
        if key not in self.entities:
            self.entities[key] = {"name": name, "type": entity_type, "facts": [], "mentions": 0}
        self.entities[key]["mentions"] += 1
        if fact and fact not in self.entities[key]["facts"]:
            self.entities[key]["facts"].append(fact)
    
    def get(self, name):
        return self.entities.get(name.lower())
    
    def format_context(self, names):
        lines = []
        for name in names:
            e = self.get(name)
            if e:
                facts = "; ".join(e["facts"][-3:]) or "no details"
                lines.append(f"- {e['name']} ({e['type']}): {facts}")
        return "\n".join(lines)

# Test
memory = EntityMemory()
memory.update("Sarah Chen", "person", "works in engineering")
memory.update("Project Atlas", "project", "deadline March 1st")
memory.update("Sarah Chen", "person", "leading backend team")

print("Sarah:", memory.get("sarah chen"))
print("\nContext:")
print(memory.format_context(["Sarah Chen", "Project Atlas"]))

---
## Exercise 13.4.3 Solution: Entity-Aware Agent

In [ ]:
from openai import OpenAI
import json

client = OpenAI()

class EntityAgent:
    def __init__(self):
        self.history = []
        self.entities = {}
    
    def extract_entities(self, text):
        try:
            resp = client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=[{"role": "user", "content": f'Extract as JSON: {{"entities": [{{"name":"...","type":"...","info":"..."}}]}} from: {text}'}],
                max_tokens=150
            )
            return json.loads(resp.choices[0].message.content).get("entities", [])
        except:
            return []
    
    def store_entities(self, text):
        for e in self.extract_entities(text):
            key = e["name"].lower()
            if key not in self.entities:
                self.entities[key] = {"name": e["name"], "type": e["type"], "facts": []}
            if e.get("info"):
                self.entities[key]["facts"].append(e["info"])
    
    def chat(self, user_input):
        if "what do you know about" in user_input.lower():
            name = user_input.lower().split("what do you know about")[-1].strip().rstrip("?")
            e = self.entities.get(name)
            if e:
                return f"I know {e['name']} is a {e['type']}. Facts: {'; '.join(e['facts']) or 'none'}"
            return f"I don't have information about '{name}'."
        
        messages = [{"role": "system", "content": "You are helpful."}] + self.history[-10:]
        messages.append({"role": "user", "content": user_input})
        
        resp = client.chat.completions.create(model="gpt-3.5-turbo", messages=messages)
        reply = resp.choices[0].message.content
        
        self.store_entities(user_input)
        self.history.extend([{"role": "user", "content": user_input}, {"role": "assistant", "content": reply}])
        return reply

# Test
agent = EntityAgent()
print(agent.chat("I'm working with Maria on the Sunrise project. She handles design."))
print(agent.chat("What do you know about Maria?"))

---
## Exercise 13.5.1 Solution: Basic Semantic Search

In [ ]:
import chromadb
from openai import OpenAI

class SemanticMemory:
    def __init__(self):
        self.openai = OpenAI()
        self.chroma = chromadb.Client()
        self.collection = self.chroma.create_collection("memories")
        self.count = 0
    
    def _embed(self, text):
        response = self.openai.embeddings.create(model="text-embedding-3-small", input=text)
        return response.data[0].embedding
    
    def add(self, text):
        self.count += 1
        self.collection.add(ids=[f"m{self.count}"], embeddings=[self._embed(text)], documents=[text])
    
    def search(self, query, n=3):
        results = self.collection.query(query_embeddings=[self._embed(query)], n_results=n)
        return results["documents"][0]

# Test
memory = SemanticMemory()
memory.add("Python is a programming language")
memory.add("The Eiffel Tower is in Paris")
memory.add("Machine learning requires data")

print("Query: 'AI and data science'")
for r in memory.search("AI and data science"):
    print(f"  - {r}")

---
## Exercise 13.5.2 Solution: Memory with Categories

In [ ]:
import chromadb
from openai import OpenAI

class CategorizedMemory:
    def __init__(self):
        self.openai = OpenAI()
        self.chroma = chromadb.Client()
        self.collection = self.chroma.create_collection("categorized")
        self.count = 0
        self.categories = {}
    
    def _embed(self, text):
        return self.openai.embeddings.create(model="text-embedding-3-small", input=text).data[0].embedding
    
    def add(self, text, category):
        self.count += 1
        self.collection.add(
            ids=[f"m{self.count}"], embeddings=[self._embed(text)],
            documents=[text], metadatas=[{"category": category}]
        )
        self.categories[category] = self.categories.get(category, 0) + 1
    
    def search(self, query, n=3):
        results = self.collection.query(query_embeddings=[self._embed(query)], n_results=n)
        return results["documents"][0]
    
    def search_category(self, query, category, n=3):
        results = self.collection.query(
            query_embeddings=[self._embed(query)], n_results=n, where={"category": category}
        )
        return results["documents"][0] if results["documents"] else []
    
    def get_category_counts(self):
        return self.categories.copy()

# Test
memory = CategorizedMemory()
memory.add("Finish report by Friday", "work")
memory.add("Buy groceries", "personal")
memory.add("App idea: habit tracker", "ideas")

print("Categories:", memory.get_category_counts())
print("Work search:", memory.search_category("deadlines", "work"))

---
## Exercise 13.5.3 Solution: Conversational Agent with Semantic Recall

In [ ]:
import chromadb
from openai import OpenAI

class SemanticAgent:
    def __init__(self, system_prompt):
        self.openai = OpenAI()
        self.system_prompt = system_prompt
        self.history = []
        self.chroma = chromadb.Client()
        self.memories = self.chroma.create_collection("agent_mem")
        self.mem_id = 0
    
    def _embed(self, text):
        return self.openai.embeddings.create(model="text-embedding-3-small", input=text).data[0].embedding
    
    def _store(self, content):
        self.mem_id += 1
        self.memories.add(ids=[f"m{self.mem_id}"], embeddings=[self._embed(content)], documents=[content])
    
    def _retrieve(self, query, n=3):
        if self.memories.count() == 0:
            return []
        results = self.memories.query(query_embeddings=[self._embed(query)], n_results=min(n, self.memories.count()))
        return results["documents"][0]
    
    def remember(self, fact):
        self._store(f"Fact: {fact}")
        return f"Stored: {fact}"
    
    def chat(self, user_input):
        relevant = self._retrieve(user_input)
        messages = [{"role": "system", "content": self.system_prompt}]
        if relevant:
            messages.append({"role": "system", "content": f"Relevant: {relevant}"})
            print(f"[DEBUG] Retrieved: {relevant}")
        messages.extend(self.history[-10:])
        messages.append({"role": "user", "content": user_input})
        
        response = self.openai.chat.completions.create(model="gpt-3.5-turbo", messages=messages)
        reply = response.choices[0].message.content
        
        self.history.extend([{"role": "user", "content": user_input}, {"role": "assistant", "content": reply}])
        self._store(f"Discussed: {user_input[:80]}")
        return reply

# Test
agent = SemanticAgent("You are helpful with good memory.")
print(agent.remember("User's favorite color is blue"))
print(agent.chat("What are my preferences?"))

---
## Exercise 13.6.1 Solution: Basic Chat with Memory

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory

store = {}

def get_session_history(session_id: str):
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are friendly."),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{input}")
])

chain = prompt | ChatOpenAI(model="gpt-3.5-turbo")
chat = RunnableWithMessageHistory(chain, get_session_history, input_messages_key="input", history_messages_key="history")
config = {"configurable": {"session_id": "demo"}}

# 3-message conversation
for msg in ["Hi! My name is Sarah.", "I'm an engineer.", "What's my name and job?"]:
    print(f"Human: {msg}")
    response = chat.invoke({"input": msg}, config=config)
    print(f"AI: {response.content}\n")

print("--- Full History ---")
for msg in store["demo"].messages:
    print(f"{msg.type}: {msg.content[:50]}...")

---
## Exercise 13.6.2 Solution: Windowed Memory with trim_messages

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.runnables import RunnableLambda
from langchain_core.messages import trim_messages
from langchain_community.chat_message_histories import ChatMessageHistory

store = {}

def get_session_history(session_id: str):
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

trimmer = trim_messages(max_tokens=100, strategy="last", token_counter=ChatOpenAI(model="gpt-3.5-turbo"), include_system=True, start_on="human")

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful. Remember details."),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{input}")
])

def trim_history(input_dict):
    history = input_dict.get("history", [])
    return {**input_dict, "history": trimmer.invoke(history)}

chain = RunnableLambda(trim_history) | prompt | ChatOpenAI(model="gpt-3.5-turbo")
chat = RunnableWithMessageHistory(chain, get_session_history, input_messages_key="input", history_messages_key="history")
config = {"configurable": {"session_id": "window"}}

# 6-message conversation
messages = ["My name is Alex.", "I live in Boston.", "I have a cat.", "I work at a startup.", "I love pizza.", "What do you remember about me?"]

for msg in messages:
    print(f"Human: {msg}")
    response = chat.invoke({"input": msg}, config=config)
    print(f"AI: {response.content}\n")

print("Early messages should be forgotten, recent ones remembered.")

---
## Exercise 13.6.3 Solution: Agent with Tools and Memory Management

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.tools import tool

@tool
def calculator(expression: str) -> str:
    """Evaluate math expressions like '2+2'."""
    try:
        return f"Result: {eval(expression)}"
    except:
        return "Error"

@tool
def get_weather(city: str) -> str:
    """Get weather for a city."""
    return {"Seattle": "Rainy, 55F", "Austin": "Sunny, 85F"}.get(city, "Partly cloudy, 70F")

class AgentWithManagedMemory:
    def __init__(self, max_messages=6, keep_recent=2):
        self.model = ChatOpenAI(model="gpt-4o").bind_tools([calculator, get_weather])
        self.summary_model = ChatOpenAI(model="gpt-3.5-turbo")
        self.max_messages = max_messages
        self.keep_recent = keep_recent
        self.messages = []
        self.summary = ""
    
    def _maybe_summarize(self):
        if len(self.messages) <= self.max_messages:
            return
        to_summarize = self.messages[:-self.keep_recent]
        text = "\n".join(f"{'User' if isinstance(m, HumanMessage) else 'AI'}: {m.content[:100]}" for m in to_summarize)
        response = self.summary_model.invoke([HumanMessage(content=f"Summarize briefly: {self.summary}\n{text}")])
        self.summary = response.content
        self.messages = self.messages[-self.keep_recent:]
        print(f"[Summarized {len(to_summarize)} messages]")
    
    def chat(self, user_input):
        context = f"Previous: {self.summary}" if self.summary else ""
        messages = [{"role": "system", "content": f"You have tools. {context}"}]
        messages.extend([{"role": "user" if isinstance(m, HumanMessage) else "assistant", "content": m.content} for m in self.messages])
        messages.append({"role": "user", "content": user_input})
        
        response = self.model.invoke(messages)
        self.messages.append(HumanMessage(content=user_input))
        self.messages.append(AIMessage(content=response.content))
        self._maybe_summarize()
        return response.content

# Test
agent = AgentWithManagedMemory(max_messages=6, keep_recent=2)
print("1.", agent.chat("What's the weather in Seattle?"))
print("2.", agent.chat("Calculate 15 * 8"))
print("3.", agent.chat("What city did I ask about?"))

---
## Congratulations!

You've completed Part 3: Building AI Agents!

Next: **Part 4: LangGraph** (Chapter 14)